# tfidf

In [1]:
import numpy as np
import pandas as pd

def sigmoid(z):
    z = np.clip(z, -500, 500)  # Clip to prevent overflow in the exponential calculation
    return 1 / (1 + np.exp(-z))

def compute_gradient(X, y, w, sigma2):
    m = X.shape[0]
    y_pred = sigmoid(np.dot(X, w)) * 2 - 1  # Transform sigmoid output to -1 and +1
    gradient = np.dot(X.T, (y_pred - y)) / m + w / sigma2  # L2 regularization
    return gradient

def logistic_regression_SGD(X_train, y_train, learning_rate, epochs, sigma2, batch_size=32):
    w = np.zeros(X_train.shape[1])  # Initialize weights to zeros
    num_samples = X_train.shape[0]
    
    for epoch in range(epochs):
        indices = np.random.permutation(num_samples)
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]
        
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X_train_shuffled[start:end]
            y_batch = y_train_shuffled[start:end]
            
            gradient = compute_gradient(X_batch, y_batch, w, sigma2)
            w -= learning_rate * gradient

        learning_rate /= (1 + epoch)  # Decaying learning rate each epoch

    return w

def predict(X, weights):
    z = np.dot(X, weights)
    return np.where(sigmoid(z) >= 0.5, 1, -1)

def calculate_metrics(y_true, y_pred):
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    false_positives = np.sum((y_pred == 1) & (y_true == -1))
    false_negatives = np.sum((y_pred == -1) & (y_true == 1))
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

def calculate_accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    return correct_predictions / total_predictions

def perform_cross_validation(X, y, k, learning_rates, sigma2_values, epochs, batch_size):
    num_samples = len(y)
    folds = k_fold_split(num_samples, k)
    best_f1 = -np.inf
    best_params = {}

    for gamma in learning_rates:
        for sigma2 in sigma2_values:
            f1_scores = []

            for i in range(k):
                test_idx = folds[i]
                train_idx = np.hstack([folds[j] for j in range(k) if j != i])

                X_train, X_valid = X[train_idx], X[test_idx]
                y_train, y_valid = y[train_idx], y[test_idx]
                
                weights = logistic_regression_SGD(X_train, y_train, gamma, epochs, sigma2, batch_size)
                y_pred_valid = predict(X_valid, weights)
                _, _, f1 = calculate_metrics(y_valid, y_pred_valid)
                
                f1_scores.append(f1)

            avg_f1 = np.mean(f1_scores)
            if avg_f1 > best_f1:
                best_f1 = avg_f1
                best_params = {'gamma': gamma, 'sigma2': sigma2}

    return best_params

def k_fold_split(num_samples, k=5):
    indices = np.arange(num_samples)
    np.random.shuffle(indices)
    fold_sizes = np.full(k, num_samples // k, dtype=int)
    fold_sizes[:num_samples % k] += 1
    current = 0
    folds = []
    for fold_size in fold_sizes:
        start, stop = current, current + fold_size
        folds.append(indices[start:stop])
        current = stop
    return folds

# Load datasets
full_train_data = pd.read_csv('/home/u1472278/ML/Project/project_data/new.tfidf.train.csv')
dev_data = pd.read_csv('/home/u1472278/ML/Project/project_data/data/tfidf/tfidf.eval.anon.csv')
test_data = pd.read_csv('/home/u1472278/ML/Project/project_data/new.tfidf.test.csv')

X_train_full = full_train_data.iloc[:, 1:].values
y_train_full = full_train_data.iloc[:, 0].values
X_dev = dev_data.iloc[:, 1:].values
y_dev = dev_data.iloc[:, 0].values
X_test = test_data.iloc[:, 1:].values
y_test = test_data.iloc[:, 0].values

# Hyperparameters and training configuration
learning_rates = [0.1, 0.01, 0.001]
sigma2_values = [0.01, 0.1, 1]
epochs = 100
batch_size = 64
k = 5  # Number of folds

# Perform cross-validation to find the best hyperparameters
best_params = perform_cross_validation(X_train_full, y_train_full, k, learning_rates, sigma2_values, epochs, batch_size)
print("Best Hyperparameters:", best_params)

# Train on the full training data using the best hyperparameters
final_weights = logistic_regression_SGD(X_train_full, y_train_full, best_params['gamma'], epochs, best_params['sigma2'], batch_size)

# Predict and evaluate on the dev set
y_pred_dev = predict(X_dev, final_weights)
dev_precision, dev_recall, dev_f1 = calculate_metrics(y_dev, y_pred_dev)
dev_accuracy = calculate_accuracy(y_dev, y_pred_dev)

predicted_labels_int = y_pred_dev.astype(int)
predicted_labels_df = pd.DataFrame(predicted_labels_int, columns=['label'])
eval_ids = pd.read_csv("data/eval.ids", header=None, names=['example_id'])

submission_df = pd.concat([eval_ids['example_id'], predicted_labels_df], axis=1)

# Save the concatenated DataFrame to a CSV file
submission_df.to_csv('submission_lr_tfidf.csv', index=False)
print(f"Dev Precision: {dev_precision:.4f}")
print(f"Dev Recall: {dev_recall:.4f}")
print(f"Dev F1 Score: {dev_f1:.4f}")
print(f"Dev Accuracy: {dev_accuracy:.4f}")

# Predict and evaluate on the test set
y_pred_test = predict(X_test, final_weights)
test_precision, test_recall, test_f1 = calculate_metrics(y_test, y_pred_test)
test_accuracy = calculate_accuracy(y_test, y_pred_test)
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


/tmp/ipykernel_501648/2612893350.py:11: RuntimeWarning: overflow encountered in true_divide
  gradient = np.dot(X.T, (y_pred - y)) / m + w / sigma2  # L2 regularization


Best Hyperparameters: {'gamma': 0.001, 'sigma2': 0.01}
Dev Precision: 1.0000
Dev Recall: 0.4010
Dev F1 Score: 0.5724
Dev Accuracy: 0.4010
Test Precision: 0.7207
Test Recall: 0.5923
Test F1 Score: 0.6502
Test Accuracy: 0.6916
